# R MNIST Model

 * Wrap an R model (using the ```caret``` library) for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * R

```bash
pip install seldon-core
```

## Train locally
 

In [11]:
!rm -f *ubyte && ./get_data.sh && Rscript train.R

--2019-04-24 16:43:55--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]   9.45M  2.56MB/s    in 4.5s    

2019-04-24 16:43:59 (2.10 MB/s) - ‘train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2019-04-24 16:43:59--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

train-labels-idx1-u 100%[===================>]  28.20K  --.-KB/s    in 0.08s   

2019-04-24 16:44:00 (346 KB/s) - ‘train-labels-id

Wrap model using s2i

In [12]:
!s2i build . seldonio/seldon-core-s2i-r:0.3 r-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/pls_2.7-1.tar.gz'
Content type 'application/x-gzip' length 4390103 bytes (4.2 MB)
downloaded 4.2 MB

* installing *source* package ‘pls’ ...
** package ‘pls’ successfully unpacked and MD5 sums checked
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (pls)

The downloaded source packages are in
	‘/tmp/RtmpSNT0ox/downloaded_packages’
Build completed successfully


In [13]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 r-mnist:0.1

5104ca6b9b9b9f8a778c46ae1c9d84a733ea63732dce128066306f7fcc49bb82


Send some random features that conform to the contract

In [14]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[0.109 0.293 0.668 0.174 0.445 0.31  0.777 0.777 0.366 0.064 0.191 0.17
  0.254 0.693 0.425 0.566 0.443 0.931 0.575 0.524 0.978 0.797 0.42  0.928
  0.246 0.996 0.433 0.438 0.807 0.419 0.567 0.202 0.12  0.158 0.055 0.471
  0.347 0.491 0.988 0.559 0.482 0.922 0.566 0.182 0.19  0.977 0.933 0.541
  0.507 0.708 0.633 0.919 0.578 0.451 0.569 0.08  0.835 0.625 0.376 0.534
  0.18  0.123 0.212 0.186 0.254 0.369 0.93  0.276 0.565 0.738 0.951 0.222
  0.651 0.584 0.549 0.145 0.891 0.65  0.861 0.584 0.573 0.992 0.34  0.252
  0.122 0.998 0.066 0.883 0.606 0.651 0.764 0.591 0.314 0.048 0.269 0.843
  0.116 0.944 0.633 0.639 0.166 0.537 0.115 0.559 0.798 0.646 0.996 0.492
  0.178 0.77  0.107 0.109 0.302 0.559 0.497 0.382 0.35  0.764 0.744 0.75
  0.46  0.507 0.143 0.673 0.123 0.    0.569 0.442 0.485 0.923 0.018 0.16
  0.4   0.99  0.958 0.718 0.373 0.161 0.263 0.989 0.782 0.685 0.363 0.072
  0.177 0.757 0.383 0.628 0.067 0.876 0.78  0.479 0.

In [15]:
!docker rm mnist_predictor --force

mnist_predictor


## Test using Minikube


In [ ]:
!minikube start --memory 4096

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [16]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-r:0.1 r-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/pls_2.7-1.tar.gz'
Content type 'application/x-gzip' length 4390103 bytes (4.2 MB)
downloaded 4.2 MB

* installing *source* package ‘pls’ ...
** package ‘pls’ successfully unpacked and MD5 sums checked
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (pls)

The downloaded source packages are in
	‘/tmp/RtmpDwBobW/downloaded_packages’
Build completed successfully


In [17]:
!kubectl create -f r_mnist_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


Wait until ready (replicas == replicasAvailable)

In [18]:
!kubectl rollout status deploy/r-mnist-deployment-r-mnist-predictor-d12e455

Waiting for deployment "r-mnist-deployment-r-mnist-predictor-d12e455" rollout to finish: 0 of 1 updated replicas are available...
deployment "r-mnist-deployment-r-mnist-predictor-d12e455" successfully rolled out


In [1]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    seldon-deployment-example --namespace seldon -p

----------------------------------------
SENDING NEW REQUEST:

[[0.042 0.143 0.444 0.009 0.57  0.229 0.198 0.534 0.339 0.835 0.463 0.584
  0.228 0.242 0.581 0.451 0.497 0.252 0.791 0.457 0.817 0.167 0.194 0.088
  0.824 0.014 0.278 0.186 0.43  0.538 0.321 0.847 0.    0.325 0.244 0.208
  0.15  0.984 0.306 0.564 0.507 0.676 0.48  0.218 0.641 0.395 0.277 0.992
  0.136 0.525 0.063 0.092 0.45  0.635 0.935 0.331 0.082 0.336 0.425 0.918
  0.823 0.878 0.496 0.931 0.137 0.24  0.631 0.707 0.142 0.783 0.615 0.203
  0.511 0.97  0.313 0.68  0.61  0.4   0.408 0.41  0.853 0.361 0.988 0.378
  0.557 0.095 0.367 0.418 0.478 0.312 0.873 0.267 0.396 0.82  0.423 0.848
  0.109 0.101 0.326 0.254 0.552 0.817 0.981 0.729 0.822 0.246 0.331 0.61
  0.921 0.208 0.633 0.635 0.246 0.552 0.43  0.158 0.469 0.729 0.212 0.524
  0.223 0.18  0.829 0.101 0.765 0.589 0.795 0.464 0.101 0.934 0.136 0.005
  0.672 0.516 0.837 0.133 0.544 0.425 0.711 0.225 0.213 0.593 0.987 0.611
  0.842 0.601 0.745 0.546 0.177 0.794 0.534 0.901 

In [ ]:
!minikube delete